In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
dbutils.widgets.text("storage_name", "safinalprojectdbs")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalogo", "catalog_dev")
dbutils.widgets.text("esquema", "bronze")

In [0]:
storage_name = dbutils.widgets.get("storage_name")
container = dbutils.widgets.get("container")
catalogo = dbutils.widgets.get("catalogo")
esquema = dbutils.widgets.get("esquema")

ruta = f"abfss://{container}@{storage_name}.dfs.core.windows.net/order_products__prior.csv"

In [0]:
df_order_products_prior = spark.read.option('header', True)\
                        .option('inferSchema', True)\
                        .csv(ruta)

In [0]:
order_products_prior_schema = StructType(fields=[StructField("order_id", IntegerType(), False),
                                     StructField("product_id", StringType(), True),
                                     StructField("add_to_cart_order", IntegerType(), True),
                                     StructField("reordered", IntegerType(), True)
])

In [0]:
df_order_products_prior_final = spark.read\
.option('header', True)\
.option("sep", ";")\
.schema(order_products_prior_schema)\
.csv(ruta)

In [0]:
order_products_prior_selected_df = df_order_products_prior_final.select(col("order_id"), 
                                                col("product_id"),
                                                col("add_to_cart_order"),
                                                col("reordered")
                                                )

In [0]:
 df_order_products_prior_final = order_products_prior_selected_df.withColumn("ingestion_date", current_timestamp())

In [0]:
df_order_products_prior_final.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema}.order_products_prior")